# Seqeuence to Sequence Learning with Neural Networks

本篇通过MindSpore实现论文[Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)中的模型。

## 概述

## 数据准备

我们本次使用的数据集为**Multi30K数据集**，它是一个大规模的图像-文本数据集，包含30K+图片，每张图片对应两类不同的文本描述：
- 英语描述，及对应的德语翻译；
- 五个独立的、非翻译而来的英语和德语描述，描述中包含的细节并不相同；

因其收集的不同语言对于图片的描述相互独立，所以训练出的模型可以更好地适用于有噪声的多模态内容。

![avatar](./assets/Multi30K.png)

首先，我们通过如下url下载数据：

In [2]:
# 训练、验证、测试数据集下载地址
urls = {
    'train': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz',
    'valid': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz',
    'test': 'http://www.quest.dcs.shef.ac.uk/wmt17_files_mmt/mmt_task1_test2016.tar.gz'
}

### 数据下载模块

数据下载模块使用`requests`库进行http请求，并通过`tqdm`对下载百分比进行可视化。此外针对下载安全性，使用IO的方式下载临时文件，而后保存至指定的路径并返回。
> `tqdm`和`requests`库需手动安装，命令如下：`pip install tqdm requests`

In [1]:
import os
import logging
import shutil
import requests
import tempfile
from tqdm import tqdm
from typing import IO
from pathlib import Path

# 指定保存路径为 `home_path/.mindspore_examples`
cache_dir = Path.home() / '.mindspore_examples'

def http_get(url: str, temp_file:IO):
    """使用requests库下载数据，并使用tqdm库进行流程可视化"""
    req = requests.get(url, stream=True)
    content_length = req.headers.get('Content-Length')
    total = int(content_length) if content_length is not None else None
    progress = tqdm(unit='B', total=total)
    for chunk in req.iter_content(chunk_size=1024):
        if chunk:
            progress.update(len(chunk))
            temp_file.write(chunk)
    progress.close()

def download(file_name:str, url: str):
    """下载数据并存为指定名称"""
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    cache_path = os.path.join(cache_dir, file_name)
    cache_exist = os.path.exists(cache_path)
    if not cache_exist:
        with tempfile.NamedTemporaryFile() as temp_file:
            http_get(url, temp_file)
            temp_file.flush()
            temp_file.seek(0)
            logging.info(f"copying {temp_file.name} to cache at {cache_path}")
            with open(cache_path, 'wb') as cache_file:
                shutil.copyfileobj(temp_file, cache_file)
    return cache_path

下载后的训练、验证、测试数据集各对应一个`tar.gz`文件，以训练数据集为例，原始的解压目录如下：
```text
training
├── train.de
└── train.en
```

In [3]:
train_path = download('train.tar.gz', urls['train'])
valid_path = download('valid.tar.gz', urls['valid'])
test_path = download('test.tar.gz', urls['test'])

100%|██████████| 66154/66154 [00:01<00:00, 54169.40B/s]


### 数据预处理

在使用数据进行模型训练等操作时，我们需要对数据进行预处理，流程如下：

1. 解压`tar.gz`文件；

2. 目前数据为句子形式的文本，需要进行分词，即将句子拆解为单独的词元（token，可以为字符或者单词）；

    - 分词可以使用`spaCy`创建分词器（tokenizer）：`de_core_news_sm`，`en_core_web_sm`，需要手动下载；

    - 分词后，去除多余的空格，统一大小写等；
    
3. 将每个词元映射到从0开始的数字索引中（为节约存储空间，可过滤掉词频低的词元），词元和数字索引所构成的集合叫做词典（vocabulary）；

4. 添加特殊占位符，标明序列的起始与结束，统一序列长度，并创建数据迭代器；

> `spaCy` 下载： `pip install spacy`

> 分词器（tokennizer）下载： `python -m spacy download de_core_news_sm`，`python -m spacy download en_core_web_sm`


#### 数据加载器（解压+分词）

In [4]:
import re
import six
import string
import tarfile
import spacy
from functools import partial

class Multi30K():
    """Multi30K数据集加载器
    
    加载Multi30K数据集并处理为一个Python迭代对象。
    
    """
    def __init__(self, path):
        self.data = self._load(path)
        
    def _load(self, path):
        def tokenize(text, spacy_lang):
            # 去除多余空格，统一大小写
            text = text.rstrip()
            return [tok.text.lower() for tok in spacy_lang.tokenizer(text)]
        
        # 加载英、德语分词器
        tokenize_de = partial(tokenize, spacy_lang=spacy.load('de_core_news_sm'))
        tokenize_en = partial(tokenize, spacy_lang=spacy.load('en_core_web_sm'))
        
        # 解压、读取Multi30K数据，并进行分词
        tarf = tarfile.open(path)
        members = {i.name.split('.')[-1]: i for i in tarf.getmembers()}
        de = tarf.extractfile(members['de']).readlines()[:-1]
        de = [tokenize_de(i.decode()) for i in de]
        en = tarf.extractfile(members['en']).readlines()[:-1]
        en = [tokenize_en(i.decode()) for i in en]

        return list(zip(de, en))
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [5]:
train_dataset, valid_dataset, test_dataset = Multi30K(train_path), Multi30K(valid_path), Multi30K(test_path)

对解压和分词结果进行测试，打印测试数据集第一组英德语文本，可以看到每一个单词和标点符号已经被单独分离出来。

In [6]:
for de, en in test_dataset:
    print(de, en)
    break

['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.'] ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


#### 词典

In [8]:
class Vocab:
    """通过词频字典，构建词典"""

    def __init__(self, word_count_dict, min_freq=1, special_tokens=['<unk>', '<pad>', '<bos>', '<eos>']):
        self.word2idx = {}
        for idx, tok in enumerate(special_tokens):
            self.word2idx[tok] = idx

        # 过滤低词频的词元
        filted_dict = {
            w: c
            for w, c in word_count_dict.items() if c >= min_freq
        }
        for w, _ in filted_dict.items():
            self.word2idx[w] = len(self.word2idx)

        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.bos_idx = self.word2idx['<bos>']  # 特殊占位符：句首
        self.eos_idx = self.word2idx['<eos>']  # 特殊占位符：句末
        self.pad_idx = self.word2idx['<pad>']  # 特殊占位符：补充字符
        self.unk_idx = self.word2idx['<unk>']  # 特殊占位符：低词频词元或未曾出现的词元

    def _word2idx(self, word):
        """单词映射至数字索引"""
        if word not in self.word2idx:
            return self.unk_idx
        return self.word2idx[word]

    def _idx2word(self, idx):
        """数字索引映射至单词"""
        if idx not in self.idx2word:
            raise ValueError('input index is not in vocabulary.')
        return self.idx2word[idx]

    def encode(self, word_or_list):
        """将单个单词或单词数组映射至单个数字索引或数字索引数组"""
        if isinstance(word_or_list, list):
            return [self._word2idx(i) for i in word_or_list]
        return self._word2idx(word_or_list)

    def decode(self, idx_or_list):
        """将单个数字索引或数字索引数组映射至单个单词或单词数组"""
        if isinstance(idx_or_list, list):
            return [self._idx2word(i) for i in idx_or_list]
        return self._idx2word(idx_or_list)

    def __len__(self):
        return len(self.word2idx)

通过自定义词频字典进行测试，我们可以看到词典已去除词频少于2的词元c，并加入了默认的四个特殊占位符，故词典整体长度为：4 - 1 + 4 = 7

In [9]:
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)

7

使用`collections`中的`Counter`和`OrderedDict`统计英/德语每个单词在整体文本中出现的频率。构建词频字典，然后再将词频字典转为词典。

在分配数字索引时有一个小技巧：常用的词元对应数值较小的索引，这样可以节约空间。

> collections 需要手动下载： `pip install collections`

In [10]:
from collections import Counter, OrderedDict

def build_vocab(dataset):
    de_words, en_words = [], []
    for de, en in dataset:
        de_words.extend(de)
        en_words.extend(en)
        
    de_count_dict = OrderedDict(sorted(Counter(de_words).items(), key=lambda t: t[1], reverse=True))
    en_count_dict = OrderedDict(sorted(Counter(en_words).items(), key=lambda t: t[1], reverse=True))
    
    return Vocab(de_count_dict, min_freq=2), Vocab(en_count_dict, min_freq=2)

In [12]:
de_vocab, en_vocab = build_vocab(train_dataset)
print('Unique tokens in de vocabulary:', len(de_vocab))

Unique tokens in de vocabulary: 7853


#### 数据迭代器

数据预处理的最后一步是创建数据迭代器，我们在进一步处理数据（包括批处理，添加起始和终止符号，统一序列长度）后，将数据以张量的形式返回。

创建数据迭代器需要如下参数：

- `dataset`：分词后的数据集
- `de_vocab`：德语词典
- `en_vocab`：英语词典
- `batch_size`：批量大小，即一个batch中包含多少个序列
- `max_len`：序列最大长度，如不满则补齐，如超过则丢弃
- `drop_remainder`：是否在最后一个batch未满时，丢弃该batch

In [72]:
import mindspore

class Iterator():
    """创建数据迭代器"""
    def __init__(self, dataset, de_vocab, en_vocab, batch_size, max_len=32, drop_reminder=False):
        self.dataset = dataset
        self.de_vocab = de_vocab
        self.en_vocab = en_vocab
        
        self.batch_size = batch_size
        self.max_len = max_len
        self.drop_reminder = drop_reminder

        length = len(self.dataset) // batch_size 
        self.len = length if drop_reminder else length + 1 # 批量数量
    
    def __call__(self):
        def pad(idx_list, vocab, max_len):
            """统一序列长度，并记录有效长度"""
            idx_pad_list, idx_len = [], []
            # 当前序列度超过最大长度时，将超出的部分丢弃；当前序列长度小于最大长度时，用占位符补齐
            for i in idx_list:
                if len(i) > max_len - 2:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i[:max_len-2] + [vocab.eos_idx]
                    )
                    idx_len.append(max_len)
                else:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i + [vocab.eos_idx] + [vocab.pad_idx] * (max_len - len(i) - 2)
                    )
                    idx_len.append(len(i) + 2)
            return idx_pad_list, idx_len

        def sort_by_length(src, trg):
            """对德/英语的字段长度进行排序"""
            data = zip(src, trg)
            data = sorted(data, key=lambda t: len(t[0]), reverse=True)
            return zip(*list(data))
            
        def encode_and_pad(batch_data, max_len):
            """将批量中的文本数据转换为数字索引，并统一每个序列的长度"""
            # 将当前批量数据中的词元转化为索引
            src_data, trg_data = zip(*batch_data)
            src_idx = [self.de_vocab.encode(i) for i in src_data]
            trg_idx = [self.en_vocab.encode(i) for i in trg_data]
            
            # 统一序列长度
            src_idx, trg_idx = sort_by_length(src_idx, trg_idx)
            src_idx_pad, src_len = pad(src_idx, de_vocab, max_len)
            trg_idx_pad, _ = pad(trg_idx, en_vocab, max_len)
            
            return src_idx_pad, src_len, trg_idx_pad
        
        for i in range(self.len):
            # 获取当前批量的数据
            if i == self.len - 1 and not self.drop_reminder:
                batch_data = self.dataset[i * self.batch_size:]
            else:
                batch_data = self.dataset[i * self.batch_size: (i+1) * self.batch_size]
            
            src_idx, src_len, trg_idx = encode_and_pad(batch_data, self.max_len)
            # 将序列数据转换为tensor
            yield mindspore.Tensor(src_idx, mindspore.int32), \
                mindspore.Tensor(src_len, mindspore.int32), \
                mindspore.Tensor(trg_idx, mindspore.int32)
    
    def __len__(self):
        return self.len

In [73]:
train_iterator = Iterator(train_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=True)
valid_iterator = Iterator(valid_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)
test_iterator = Iterator(test_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)

In [74]:
len(train_iterator)

226

In [79]:
for src_idx_pad, src_len, trg_idx_pad in train_iterator():
    print(src_idx_pad.shape, src_len.shape)
    break

(128, 32) (128,)


## 构建模型

### 编码器（Encoder）

论文中的实现使用了4层的LSTM，在这里我们将其简化为2层。

在一个多层循环神经网络（multi-layer RNN），输入语句$X$会首先通过embedding层转换为向量，随后进入RNN模型的第一层以及后续的隐藏状态中，最后输出